In [ ]:
# def process_data(input_file, output_file):
#     with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
#         for line in infile:
#             # Split the line into its components
#             parts = line.strip().split(',')
            
#             # Check if we have at least 5 parts (file path + 4 features)
#             if len(parts) >= 5:
#                 # Replace 'None' with 'mixed' in the 4th column (3rd feature)
#                 if parts[3] == 'None':
#                     parts[3] = 'mixed'
                
#                 # Replace 'None' with 'unknown' in other columns
#                 for i in range(1, len(parts)):
#                     if i != 3 and parts[i] == 'None':
#                         parts[i] = 'unknown'
            
#             # Join the parts back into a line and write to output file
#             outfile.write(','.join(parts) + '\n')

# # Usage
# input_file = '/home/zakaria/workspace/ai-photo-enhancer/dataset/categories.txt'
# output_file = 'processed_data.txt'
# process_data(input_file, output_file)

In [4]:
from torch.utils.data import Dataset
from torchvision.io import read_image
import torchvision.transforms.v2.functional as F
import random
import os
from pathlib import Path



# torch.inference_mode():
class FiveKDataset(Dataset):
    """
    A dataset that reads Adobe5K dataset images
    output : tensor of unprocessed images
    """
    def __init__(self, image_size,mode="train", resize= True,augment_data=True):
        current_dir = os.getcwd()
        dataset_dir = os.path.join(current_dir, "..", "dataset", "FiveK")
        
        if mode =='train':
            self.IMGS_PATH = os.path.join(dataset_dir, "train")
        else:
            self.IMGS_PATH = os.path.join(dataset_dir, "test")
        self.resize= resize
        self.image_size =image_size
        self.augment_data = augment_data
        self.img_files = [filename for filename in os.listdir(self.IMGS_PATH+'/input/')]

    def __len__(self):
        return len(self.img_files)
    
    def __getitem__(self,idx):
        source_path = self.img_files[idx]

        source = read_image(self.IMGS_PATH+'/input/'+source_path)
        target = read_image(self.IMGS_PATH+'/target/'+source_path)
        if self.resize:
            source =  F.resize(source,(self.image_size,self.image_size), interpolation= F.InterpolationMode.BICUBIC)
            target =  F.resize(target,(self.image_size,self.image_size), interpolation= F.InterpolationMode.BICUBIC)
            
        if self.augment_data:
            if random.random() > 0.5:
                source = F.hflip(source)
                target = F.hflip(target)
            if random.random() > 0.5:
                source = F.vflip(source)
                target = F.vflip(target)
            
        # source = self.transform(Image.open(ORIGINAL_FOLDER+source_path))

        return source, target
